In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
    FROM
    (SELECT id, tt_ad
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
    WHERE (tt_ad='') is false) as d;
    """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(llt_name))) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
ads = pd.DataFrame(get_total_ads())

In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['name']

In [7]:
ads['refined'] = ''
ads.head()

,name,refined
0,mass-cavity,
1,anuloaortic ectasia,
2,presyncope,
3,seizure,
4,proliferative diabetic retinopathy,


#### 전처리 전 총 부작용의 수는 9282개 입니다

In [8]:
ads.shape

(9282, 2)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
dictionary = get_ad_dictionary()
dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
len(dictionary)

60939

In [11]:
dictionary[:10]

['ventilation pneumonitis',
 '11-beta-hydroxylase deficiency',
 '11-oxysteroid activity incr',
 '11-oxysteroid activity increased',
 '17 ketosteroids urine abnormal',
 '17 ketosteroids urine abnormal nos',
 '17 ketosteroids urine decreased',
 '17 ketosteroids urine high',
 '17 ketosteroids urine increased',
 '17 ketosteroids urine low']

In [12]:
ads.sort_values(by=['name'])[350:400]

,name,refined
3902,ageshock,
6640,ageusia,
7246,aggression,
6297,aggressive behavior,
2512,aggressive behaviour,
3973,aggressiveness,
4083,agitated,
4876,agitated depression,
620,agitation,
3200,agitation neonatal,


In [13]:
ads.loc[ads.name.str.find('beverage')>=0]
# ads.loc[ads.name.str.endswith('-related')]

,name,refined
83,non-beverage,
1280,beverage,
8768,beverages,


#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

In [14]:
replace_with_space = ['<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific']

replace_ends_with = ['-related']

In [15]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind]['refined'] = row['name'].replace(end, '')

ads = ads.reset_index(drop=True)

mass-cavity
anuloaortic ectasia
presyncope
seizure
proliferative diabetic retinopathy
aphthous ulcer
post-aids
shoulder pain
granuloma
gallbladder stones
ventricular dyssynchrony
costochondritis
splenic thrombosis
sleep onset paralysis
epispadias-classical
vascular access complication
presyncopes
iatrogenic injury
detrusor hyperreflexia
back pain
delusional perception
infarction-free
microneutralisation
sunken eyes
sirs
broncho-pneumonia
hypermagnesaemia
conjunctival laceration
hemorrhagic conversion stroke
alhadh includes
intussusception
idiopathic short stature
carcinoma of lung
anencephaly
lbbb
enterococcussepsis
myeloid sarcoma
cytomegalovirus infection
nonfluorinated
gliosis
antitachycardia
cardio-myopathy
cardiac dysrhythmias
infarcton
onychoclasis
vocal tic
primary hypercholesterolemia
sepsis-related
subendocardial fibrosis
familia
sensory disturbance
hypertension-stroke
heart insufficiency
nyctalopia
tick bite
low ph
chromophobe renal cell carcinoma
systemic candidaand
ketosis


human immunodeficiency virus type
sarcoma of skin
vertigo positional
immune system disorders
menstrual irregularity
eosinophilic granulomatosis with polyangiitis
groin haematoma
pins and needles
high-pressure
macroglossia
multiple sclerosis lesions
anterior uveitis
protein c deficiency
melanoma in situ
atrophy
diarrhea
myelopathy
transplant failure
esophageal cancer metastatic
partial seizures
retinopathy background
solar lentigo
constitutional syndrome
urinary-tract abnormalities
side branch loss
pseudo-gout
greyscale
anisocytosis
avian influenza
conjunctival oedema
hair thinning
peri-procedural complications
post-endocarditis
hsil
modulation-blind
cholecystectomies
substance-induced psychotic disorder
spondylosis
pituitary disorders
lacerations
hepatitis b-haemophilus
medication overuse headache
low back pain
electrocution
palmar-plantar erythrodysaesthesia syndrome
chronic bronchitis
other paralytic syndromes
lipoatrophy
bursitis
bachelor's
fetal heart rate abnormalities
peripheral 

confusional state
hairy cell leukaemia
muscular pain
retinal haemorrhage
situs inversus
ventricular ectopics
copd exacerbation
hyperkalemia
loss of interest
proctalgia
thrombosis
soft tissue sarcoma
condyloma
atopic keratoconjunctivitis
gastroduodenal ulcer
acute dyspnoea
lvaneurysm
congenital strabismus
-amyloidosis
perianal lesion
diamond-blackfan anaemia
cerebral white matter lesion
normotension
multiple organ dysfunction syndrome
other nutritional deficiencies
trisomy 9
prostatic adenoma
fibroma
breech delivery
intestinal dilatation
autoimmune thyroiditis
retrocollis
hypertransaminasaemia
adhesions
caecal polyp
unwanted pregnancy
aortic incompetence
plastic bronchitis
deaths÷ili
pustule
throat swelling
central nervous system infection
inflammation—eg
atrophie blanche
partial anomalous pulmonary venous connection
wilms tumor
breast discomfort
counterpressure
dizziness postural
embolic stroke
alcohol use disorder
dry eye
hiv-1-infected
emphysema−950
neutralising antibodies negative
s

reaction skin
signal-
blood pressure higher
death–progressive
complex regional pain syndrome
bilateral pneumonia
aplastic anemia
catheter thrombosis
accidental poisoning
other endocrine disorders
breast induration
prolonged rupture of membranes
idiopathic generalised epilepsy
thyroid carcinoma
autonomic neuropathy
osteosarcoma
ataxic
necrosis
peripheral neuropathy
av block
orthostatic hypotension
multivariable+
oropharyngeal pain
petechia
pneumonia cytomegaloviral
metastatic pain
hiv-1
hemimegalencephaly
cytomegalovirus encephalitis
recording
post-polypectomy
vestibular disorder
teratomas
keratopathy
nonanemic
embryonal rhabdomyosarcoma
timothy syndrome
dysmyelination
-atrial septal defect
wound complication
procedural complication
tremor-dominant
dystonia-parkinsonism
drug-induced psychosis
hydatidiform mole
hyperintensities
gastric haemorrhage
stress-resting
postmononucleosis
hypoestrogenemia
electrocardiogram abnormal
blood glucose decreased
raised inr
non-typhoid
expanded disabilit

meniscus lesion
hand
hiv negative
-mitral regurgitation
broken hip
hashimoto's thyroiditis
contusion of back
filariasis
hand and foot syndrome
headaches
atrial arrhythmias
scale–catastrophising
pain in limb
mlh1 mutation
chilblains
diabetes-related
pancreatic neuroendocrine tumour
choriocarcinoma
rheumatoid factor positive
liver disorder
von hippel-lindau disease
rectal cancer stage iv
ventriculitis
ischaemia cerebrovascular
windscale
tingling
bowel cancer
dependence
eye swelling
postoperative nausea
foreign body in eyes
chickenpox
non-dengue
motor tic
wheezy bronchitis
developmental delays
embolus
retained placenta
vomiting
mri abnormalities
retention of urine
acromegaly
plasma cell disorder
malabsorption
heart failure with preserved ejection fraction
burning sensation
eye discharge
septicaemia
abnormal pregnancy
myocardial necrosis
atypical ductal hyperplasias
salivary hypersecretion
systemic mastocytosis
metabolic acidosis
blighted ovum
calculus urinary
ischemic ulcers
weight decrea

...

lymphopenia
diarrhoeal
virilisation
paranoid schizophrenia
-left bundle branch block
mass—non-seminoma
pulmonary hypoplasia
gastrointestinal polyps
abdominal lymphadenopathy
skin lesion
tension-
transaminases increased
melanomas
mental impairment
behavioural and psychiatric symptoms of dementia
christensen
heart attacks
tooth infection
abdominal pain lower
negative pregnancy test
neurodegenerative disorder
co-infection
herpes infections
anomalous pulmonary venous connection
vulval cancer
dangerous
liver function test abnormality
lip smacking
protein-calorie malnutrition
lymphatic disorders
rigor
hypertension-1
sinus bradycardia
coloboma
leukaemia relapse
kalache
endoscopically
pterygium
general physical health deterioration
bleeding tendency
radiosurgery
atrial ectopic tachycardia
mycobacterium avium complex infection
tibia fracture
gastrointestinal bleed
anoxic brain damage
thalassemia
congenital glaucoma
chemical injury
flutter/fib
pre-hyper
carcinoma of oesophagus
fetal demise
phosp

ventricular arrhythmias
sudep
postdischarge
muscle-manifesting
muscle strain
body–mass
hyperenhancement
multiple endocrine neoplasia
skin ulcer
renal artery stenosis
chronic psychosis
mass-treatment
premature labour
painful joints
renal calculus
humiliating?
hyper-insulinaemia
contusion of abdominal wall
acute epiglottitis
tongue discolouration
variances
congestive heart failure-bb
pneumopericardium
hypoaesthesia oral
trismus
leukoplakia
mood altered
suicide attempt
marginal zone lymphoma
sleep disturbance
pancolitis
extension
gravidarum
acute and transient psychotic disorder
capnothorax
migraine without aura
muckle-wells syndrome
implant site pain
sertrali
vaginal laceration
chronic obstructive lung disease
emotional disorders
abscesses
pulmonary vein occlusion
bifascicular block
liver abscess
arterial occlusion
olfactory hallucinations
gonadotropin deficiency
welts
postoperative pain
sleep problem
haemodynamic instability
constrained
non-granulomatous
venous thromboembolism
arthropod

diabetes.
maculopathy
ulcer bleeding
scarf
escherichia sepsis
cerebral abscesses
mediastinal mass
migraine aura
toe walking
multicystic dysplastic kidneys
parasomnia
infarction-11b
bullous keratopathy
azoospermia
laryngomalacia
prolactinoma
chronic fever
obesity-hypoventilation
internal herniation
cannabis dependence
breast calcifications
urinary retention
difficulty sleeping
flushing
inflammatory reaction
fibrinogen decreased
bannayan-riley-ruvalcaba syndrome
granular cell tumour
chills and fever
cardiac deaths
aseptic meningitis
backache
onycholysis
pressure-dimension
renal calculi
foreign body sensation in eyes
type i diabetes mellitus
dyspnea-limited
carotid bruit
swollen ankles
n-stemi
microscopic polyangiitis
epicanthal
anorexia
agus
bell's palsy
mediastinitis
stressful
pollakiuria
dermatitis bullous
vaccinia-naive
rrageis
on-treatment failures
nonmelanoma
blood in stools
cow's milk allergy
renal cyst
alcoholic liver disease
type iii hyperlipidemia
middle cerebral artery stenosis

lymphoid leukaemia
gum hyperplasia
neuroborreliosis
peripartum cardiomyopathy
abdominal tenderness
antepartum hemorrhage
cholerae
prolonged pregnancy
ragusa
calcium deposits
thoracic aortic aneurysm
sore throat
crp increased
acute schizophrenia
glucose increased
posterior subcapsular cataract
headache
hyperkinesia
pre-diabetes
emilia
hypoglycaemic unconsciousness
carcinoma-in-situ
acute pulmonary oedema
i.afib
ischemia-driven
superior vena cava syndrome
non-psychotic disorders
labour induced
therapy naive
osteoarthrosis
resting
chest wall pain
hypercortisolism
melaena
brain stem dysfunction
pelvic fibrosis
psychopathology
depressive illness
antihypertensives
chest-pain
change in bowel habits
vaginal discharge
bacteremia
discharge
hypoparathyroidism
iron deficiency anaemia
chronotropic incompetence
vulvovaginal dryness
granulocytopenia
postmenopausal haemorrhage
cholecystitis chronic
biventricular dysfunction
osteoporosis-related
non-ischemia-guided
plasmocytosis
nutcracker oesophagus
b

chronic atrial fibrillation
cystourethrography
metastatic neoplasm
historyshock
primary transmission
renal stone
subaortic obstruction
post-bariatric
postpartum bleeding
behavioural disorders
diabetic hyperosmolar coma
chapped skin
hyperfractionation
pupillary reflex absent
hypomagnesaemia
alternans
uterine leiomyosarcoma
cold extremities
ischemia-reperfusion
hyperexcitability
american-australian-african
cardiac index increased
co-testing
decubitus ulcer
ventricular escape rhythms
stenosis
nightmare
transposition of great vessels
macular fibrosis
noninsulin-dependent diabetes mellitus
increased tendency to bruise
inflammatory arthritis
sialoadenitis
dysrhythmias
infant-testing
scrotal hernia
panniculitis-like
panniculitis 
alzheimer's disease
intestinal hemorrhage
qt prolonged
truncal obesity
vaginal delivery
asthma–eosinophilic
-pressure
nodal blocking
persistent fetal circulation
face injury
feeding disorder neonatal
arm paralysis
perineal abscess
low weight
multiple injuries
cogniti

#### 전처리와 약물이 아닌 레코드를 제거하면 9281개가 남습니다

In [16]:
ads.shape

(9275, 2)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [17]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [18]:
for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads = ads.reset_index(drop=True)

infarction-free
covariable
multivariable–adjusted
unit-variable
arrhythmia-free.
infarct-free
multivariable
4-variable
univariable
sepsis-free
multivariable+
variables.
seizure-free
dependent-variable
wrong drug administered
exercise-induced bronchospasm
pain-free
arrhythmia-free
variables
units–variable
variable:
tension-free
variable;
bivariable
cancer-free
delirium-free
diabetes-free
metastasis-free
(arrhythmia-free)
coma-free
pain-progression-free
thyroxine free decreased
leukaemia-free
variable
variablesabove
myocardial infarction-free
convulsion-free
angina-free
ventricular arrhythmia-free
stroke-free


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 9242 남습니다

In [19]:
ads.shape

(9235, 2)

In [20]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

for ind, row in ads.loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = ''.join(processed)
# comp_ads.refined = comp_ads.name.map(lambda x: remove_comparison(x))

In [21]:
ads.loc[ads.name.str.find('<')>=0]

,name,refined
406,values<02,values
654,values<0.2,values
766,stenosis<50,stenosis
1126,<asc-h,asc-h
2168,pressure<80,pressure
2892,thrombocytopenia<50,thrombocytopenia
3153,diabetes<5,diabetes
3315,values<15,values
4314,values<00001,values
4638,binge drinking<1,binge drinking


여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [22]:
additional_ads = []

for ind, row in ads.loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
ads.drop_duplicates(inplace=True)

transient ischaemic attack
amaurosis
retinal infarction
transient
healthy
retinal
amaurosis
transient
healthy
amaurosis fugax


In [23]:
ads.shape

(9244, 2)

In [24]:
ads.loc[ads.name.str.find('>')>=0]

,name,refined
755,infarction>transient ischaemic attack>amaurosis,infarction
1551,stroke>retinal infarction>transient,stroke
1743,non-epileptic seizures>healthy,non-epileptic seizures
2627,stroke>retinal,stroke
3742,attack>amaurosis,attack
4050,infarction>transient,infarction
6905,seizures>healthy,seizures
7661,strain>,strain
8190,attack>amaurosis fugax,attack
0,infarction>transient ischaemic attack>amaurosis,transient ischaemic attack


### 앞 뒤의 -, . 제거

In [25]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '-associated']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [26]:
ads.sort_values(by='name').head()

,name,refined
6799,(arrhythmia),arrhythmia
7526,(copd),copd
3511,(lbbb),lbbb
8861,(scale,scale
8071,(systemic,systemic


### 추가 전처리 필요없는 부작용들은  refined로 입력

In [27]:
remove_list.extend([')', '('])
spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
sep_pattern = re.compile("|".join(spe_chars.keys()))
int_pattern = re.compile('[0-9]')

for ind, row in ads.iterrows():
    refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
    int_refined = int_pattern.search(row['name'])
    
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() == refined.strip() and int_refined is None:
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    print('===============================')

anuloaortic ectasia
1             
1    amaurosis
Name: refined, dtype: object
presyncope
2                      
2    retinal infarction
Name: refined, dtype: object
seizure
3             
3    transient
Name: refined, dtype: object
proliferative diabetic retinopathy
4           
4    healthy
Name: refined, dtype: object
aphthous ulcer
5           
5    retinal
Name: refined, dtype: object
shoulder pain
7             
7    transient
Name: refined, dtype: object
granuloma
8           
8    healthy
Name: refined, dtype: object
gallbladder stones

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]



9                   
9    amaurosis fugax
Name: refined, dtype: object
ventricular dyssynchrony
ventricular dyssynchrony
costochondritis
costochondritis
splenic thrombosis
splenic thrombosis
sleep onset paralysis
sleep onset paralysis
vascular access complication
vascular access complication
presyncopes
presyncopes
iatrogenic injury
iatrogenic injury
detrusor hyperreflexia
detrusor hyperreflexia
back pain
back pain
delusional perception
delusional perception
microneutralisation
microneutralisation
sunken eyes
sunken eyes
sirs
sirs
hypermagnesaemia
hypermagnesaemia
conjunctival laceration
conjunctival laceration
hemorrhagic conversion stroke
hemorrhagic conversion stroke
alhadh includes
alhadh includes
intussusception
intussusception
idiopathic short stature
idiopathic short stature
carcinoma of lung
carcinoma of lung
anencephaly
anencephaly
lbbb
lbbb
enterococcussepsis
enterococcussepsis
myeloid sarcoma
myeloid sarcoma
cytomegalovirus infection
cytomegalovirus infection
nonfluorinated

restrained
alterations
alterations
cardiac murmur
cardiac murmur
nonproliferative diabetic retinopathy
nonproliferative diabetic retinopathy
neonatal hypoglycaemia
neonatal hypoglycaemia
nail psoriasis
nail psoriasis
aphonia
aphonia
duodenal perforation
duodenal perforation
addiction
addiction
cystic adenomatoid malformation
cystic adenomatoid malformation
raltegravir
raltegravir
pancytopenia
pancytopenia
nosocomial infection
nosocomial infection
acute myocardial infarction
acute myocardial infarction
drug overdose
drug overdose
developmental delay
developmental delay
myocardial degeneration
myocardial degeneration
female genital tract fistula
female genital tract fistula
tumour flare
tumour flare
hepatitis adverse
hepatitis adverse
opioid toxicity
opioid toxicity
neutropenic sepsis
neutropenic sepsis
schizophreniform disorder
schizophreniform disorder
chronic lymphoid leukaemia
chronic lymphoid leukaemia
systemic candidiasis
systemic candidiasis
incisional hernia
incisional hernia
exe

coronary heart disease
coronary heart disease
cough increased
cough increased
chronic graft versus host disease
chronic graft versus host disease
hypointense
hypointense
abnormal reflexes
abnormal reflexes
focal motor seizures
focal motor seizures
interstitial lung disease
interstitial lung disease
artery dissection
artery dissection
antidiabetes
antidiabetes
bilateral hydronephrosis
bilateral hydronephrosis
lung infiltration
lung infiltration
mouth dryness
mouth dryness
oculomotor nerve disorder
oculomotor nerve disorder
glomerulonephritis rapidly progressive
glomerulonephritis rapidly progressive
leg discomfort
leg discomfort
coagulopathy
coagulopathy
right aortic arch
right aortic arch
ganglioglioma
ganglioglioma
productive cough
productive cough
manic episodes
manic episodes
typhoid
typhoid
volvulus
volvulus
clear cell renal cell carcinoma
clear cell renal cell carcinoma
epiretinal membrane
epiretinal membrane
aortic coarctation
aortic coarctation
proliferative retinopathy
prolifer

...

anaphylaxis
anaphylaxis
burkitt's lymphoma
burkitt's lymphoma
dysthermia
dysthermia
hyperuricemia
hyperuricemia
thrombolytics
thrombolytics
haemoconcentration
haemoconcentration
topical
topical
quadraparesis
quadraparesis
dextrocardia
dextrocardia
placenta praevia
placenta praevia
african trypanosomiasis
african trypanosomiasis
epicanthus
epicanthus
hyperphosphorylated
hyperphosphorylated
fluid retention
fluid retention
gagging
gagging
fever—both
fever—both
hepatitis cirrhosis
hepatitis cirrhosis
chronic renal failure
chronic renal failure
ecg abnormalities
ecg abnormalities
oesophageal reflux
oesophageal reflux
static encephalopathy
static encephalopathy
gravid
gravid
premature beats
premature beats
leukaemia
leukaemia
urogenital disorders
urogenital disorders
anergyon
anergyon
stress
stress
oesophagus
oesophagus
haemotympanum
haemotympanum
myopic choroidal neovascularisation
myopic choroidal neovascularisation
reciprocating tachycardia
reciprocating tachycardia
dravet syndrome
dravet

apoptosis
intramyocardial bridging
intramyocardial bridging
oligoarthritis
oligoarthritis
co
co
heterotaxia
heterotaxia
hematochezia
hematochezia
pes planus
pes planus
diabetic
diabetic
psychotic
psychotic
locally advanced breast cancer
locally advanced breast cancer
canal polyps
canal polyps
paranoid delusions
paranoid delusions
temporal arteritis
temporal arteritis
comers
comers
left ventricular dyssynchrony
left ventricular dyssynchrony
vasa praevia
vasa praevia
leptospirosis
leptospirosis
low mood
low mood
thyroid adenoma
thyroid adenoma
hyperplastic
hyperplastic
inverted t waves
inverted t waves
acute leukaemia
acute leukaemia
metastasised
metastasised
myasthenia gravis crisis
myasthenia gravis crisis
lentigo
lentigo
acute renal insufficiency
acute renal insufficiency
viral arthritis
viral arthritis
dyshidrotic eczema
dyshidrotic eczema
scared
scared
autoimmune haemolytic anaemia
autoimmune haemolytic anaemia
endotracheal
endotracheal
live births
live births
pallor
pallor
unblindi

primary transmission
renal stone
renal stone
subaortic obstruction
subaortic obstruction
postpartum bleeding
postpartum bleeding
behavioural disorders
behavioural disorders
diabetic hyperosmolar coma
diabetic hyperosmolar coma
chapped skin
chapped skin
hyperfractionation
hyperfractionation
pupillary reflex absent
pupillary reflex absent
hypomagnesaemia
hypomagnesaemia
alternans
alternans
uterine leiomyosarcoma
uterine leiomyosarcoma
cold extremities
cold extremities
hyperexcitability
hyperexcitability
cardiac index increased
cardiac index increased
decubitus ulcer
decubitus ulcer
ventricular escape rhythms
ventricular escape rhythms
stenosis
stenosis
nightmare
nightmare
transposition of great vessels
transposition of great vessels
macular fibrosis
macular fibrosis
increased tendency to bruise
increased tendency to bruise
inflammatory arthritis
inflammatory arthritis
sialoadenitis
sialoadenitis
dysrhythmias
dysrhythmias
scrotal hernia
scrotal hernia
alzheimer's disease
alzheimer's disea

gallstones
gallstones
hypercholesterol
hypercholesterol
bradycardiaprolonged
bradycardiaprolonged
large for gestational age
large for gestational age
paraparesis
paraparesis
bronchopulmonary aspergillosis
bronchopulmonary aspergillosis
perinatal death
perinatal death
pregnancy
pregnancy
small for gestational age
small for gestational age
crbbb
crbbb
hepatomegaly
hepatomegaly
acute tubular necrosis
acute tubular necrosis
heart malformation
heart malformation
thrombocytopaenia
thrombocytopaenia
eye naevus
eye naevus
red urine
red urine
dead on arrival
dead on arrival
inappropriate antidiuretic hormone secretion
inappropriate antidiuretic hormone secretion
acute infection
acute infection
atypical glandular cells of undetermined significance
atypical glandular cells of undetermined significance
dysthymic disorder
dysthymic disorder
ovarian cysts
ovarian cysts
postextrasystolic
postextrasystolic
resting tremor
resting tremor
clubbing
clubbing
steatohepatitis cirrhosis
steatohepatitis cirrho

crawling
iga nephropathy
iga nephropathy
solid tumor
solid tumor
pigeon chest
pigeon chest
lumbar radiculopathy
lumbar radiculopathy
arterial perforation
arterial perforation
subclavian steal syndrome
subclavian steal syndrome
fibrillation
fibrillation
diastolic blood pressure increased
diastolic blood pressure increased
fear of death
fear of death
normal pressure hydrocephalus
normal pressure hydrocephalus
adiponectin increased
adiponectin increased
nasal obstruction
nasal obstruction
papillomavirus
papillomavirus
hernia
hernia
immune reconstitution inflammatory syndrome
immune reconstitution inflammatory syndrome
chills
chills
plaque shift
plaque shift
erythrocyte sedimentation rate high
erythrocyte sedimentation rate high
pyelonephritis
pyelonephritis
reinfection
reinfection
dermal cyst
dermal cyst
chronic headaches
chronic headaches
ly
ly
mumps
mumps
brain abscess
brain abscess
vaginal mucositis
vaginal mucositis
neurofibrillary tangles
neurofibrillary tangles
bacterial resistance


In [28]:
ads.loc[ads.refined.str.len()>0].shape

(8271, 2)

In [29]:
ads.reset_index(drop=True)

,name,refined
0,mass-cavity,
1,anuloaortic ectasia,
2,presyncope,
3,seizure,
4,proliferative diabetic retinopathy,
5,aphthous ulcer,
6,post-aids,
7,shoulder pain,
8,granuloma,
9,gallbladder stones,


## 정체된 목록입니다.

In [30]:
ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분
- 

In [31]:
# - 표시
ads[(ads['refined']=='') & (ads['name'].str.find('-')>=0)].shape

(872, 2)

In [32]:
ads.loc[ads.name.str.find('specific')>=0]

,name,refined
738,vaccinia-specific,vaccinia
1299,diabetes-specific,diabetes
2067,breast-cancer-specific,breast-cancer
2955,breast cancer-specific,breast cancer
3229,specific phobias,specific phobias
3958,stenosis-specific,stenosis
4566,hypertension-specific,hypertension
4987,cancer-specific,cancer
5376,nonspecific chest pain,nonspecific chest pain
6018,melanoma-specific,melanoma


In [ ]:
ads[(ads['refined']=='') & (ads['name'].str.find('-')>=0)][100:150]

- 추가적으로 정제하는데 어려움을 겪고 있는 약물용어가 450개 정도 됩니다. 한번 봐주시면서 어떤 접근 방식이 좋을 지 알려주시면 감사하겠습니다.

In [ ]:
drugs[drugs['refined'] == ''].shape

In [ ]:
drugs[drugs['refined'] == ''].to_csv('./need_to_refined.csv', sep=',')

In [ ]:
drugs.describe()

In [ ]:
0405-partenariat	
18f-2-fluoro-2-deoxyglucose	
18f-6-fluoro-l-dopa	
18f-fluoride	
18f-fluorodeoxyglucose	
18fluorodeoxyglucose	
1apical	
1injury

이런 애들 전처리